In [69]:
import lightgbm as lgb
import pandas as pd
import datetime

%cd C:\Users\user\Desktop\Git\Lightgbm
data = pd.read_csv('med_close.csv')


C:\Users\user\Desktop\Git\Lightgbm


In [70]:
past_1_month_data_2020=None # training data
path_3_month_data_2020=None # training data
path_1_year_data_2020=None # training data
label=None # label

validation_1_month_2021=None # validation set
validation_3_month_2021=None # validation set
validation_1_year_2021=None # validation set

In [81]:
# Split Month column by delimiter - for year, month and date
splitted_column_by_year_month_date=data['Month'].str.split('-', expand=True) 
splitted_column_by_year_month_date

# get the boolean array for Jan-2020
boolean_january_2020 = (data['Month']==datetime.datetime(2020, 1, 1).strftime('%Y-%m-%d')) | (data['Month']==datetime.datetime(2020, 2, 1).strftime('%Y-%m-%d'))# get the boolean array for Jan-2020 and Feb-2020
past_1_month_data_2020=data.loc[boolean_january_2020]

# get the boolean array for Jan-2020 to March-2020
boolean_january_to_march_2020 = (splitted_column_by_year_month_date[0]== '2020')&((splitted_column_by_year_month_date[1]== '01') | (splitted_column_by_year_month_date[1]== '02') |(splitted_column_by_year_month_date[1]== '03'))
path_3_month_data_2020 = data.loc[boolean_january_to_march_2020]

# get the boolean array for whole 2020 year
boolean_1_year_data_2020 = (splitted_column_by_year_month_date[0]== '2020')
path_1_year_data_2020=data.loc[boolean_1_year_data_2020]

# get the boolean array for Jan-2021
boolean_january_2021 = (data['Month']==datetime.datetime(2021, 1, 1).strftime('%Y-%m-%d'))# get the boolean array for Jan-2021
past_january_2021=data.loc[boolean_january_2021]

In [98]:
# Get the training data for year 2020 and label for Jan-2021
# define return as the difference between med_close

tickers = past_1_month_data_2020['Ticker'].unique() # get a list of tickers

Ticker=[]
past_one_month_data_2020=[]
past_three_month_data_2020=[]
past_one_year_data_2020=[]
label=[]

for ticker in tickers:
    Ticker.append(ticker.split('.')[0])   
    
    retrn_one_month=past_1_month_data_2020.loc[(past_1_month_data_2020['Ticker']==ticker) & (past_1_month_data_2020['Month']=='2020-02-01')]['Median'].iloc[0] -  past_1_month_data_2020.loc[(past_1_month_data_2020['Ticker']==ticker) & (past_1_month_data_2020['Month']=='2020-01-01')]['Median'].iloc[0]
    past_one_month_data_2020.append(retrn_one_month)
    
    retrn_three_month=path_3_month_data_2020.loc[(path_3_month_data_2020['Ticker']==ticker) & (path_3_month_data_2020['Month']=='2020-03-01')]['Median'].iloc[0] -  path_3_month_data_2020.loc[(path_3_month_data_2020['Ticker']==ticker) & (path_3_month_data_2020['Month']=='2020-01-01')]['Median'].iloc[0]
    past_three_month_data_2020.append(retrn_three_month)
    
    retrn_one_year=path_1_year_data_2020.loc[(path_1_year_data_2020['Ticker']==ticker) & (path_1_year_data_2020['Month']=='2020-12-01')]['Median'].iloc[0] -  path_1_year_data_2020.loc[(path_1_year_data_2020['Ticker']==ticker) & (path_1_year_data_2020['Month']=='2020-01-01')]['Median'].iloc[0]
    past_one_year_data_2020.append(retrn_one_year)
    
    # label is the med_close at Jan-2021 (assume that I am now at Dec-2020 and wants to predict Jan-2021)
    retrn_jan_2021=past_january_2021.loc[(past_january_2021['Ticker']==ticker) & (past_january_2021['Month']=='2021-01-01')]['Median'].iloc[0]
    label.append(retrn_jan_2021)

training_data=pd.DataFrame({
    'Ticker':Ticker,
    'past_one_month_data_2020':past_one_month_data_2020,   
    'past_three_month_data_2020':past_three_month_data_2020,
    'past_one_year_data_2020':past_one_year_data_2020,
    'label':label
    })
training_data
training_data.to_csv('training_data.csv')

In [100]:
# Model building
import numpy as np
model = lgb.LGBMRegressor()
data=pd.read_csv('training_data.csv')
ticker=data['Ticker'].to_numpy()
past_one_month_data_2020=data['past_one_month_data_2020'].to_numpy().astype(float)
past_three_month_data_2020=data['past_three_month_data_2020'].to_numpy().astype(float)
past_one_year_data_2020=data['past_one_year_data_2020'].to_numpy().astype(float)
label=data['label'].to_numpy().astype(int)

features = list(zip(ticker,past_one_month_data_2020, past_three_month_data_2020,past_one_year_data_2020))# Combine all the features into single list of tuples

model.fit(features, label)

LGBMRegressor()

In [ ]:
# Model testing

Ticker=[]
past_one_month_data_2021=[]
past_three_month_data_2021=[]
past_one_year_data_2021=[]
label_test=[]

for ticker in tickers:
    Ticker.append(ticker.split('.')[0])   
    
    retrn_one_month=past_1_month_data_2021.loc[(past_1_month_data_2020['Ticker']==ticker) & (past_1_month_data_2020['Month']=='2020-02-01')]['Median'].iloc[0] -  past_1_month_data_2020.loc[(past_1_month_data_2020['Ticker']==ticker) & (past_1_month_data_2020['Month']=='2020-01-01')]['Median'].iloc[0]
    past_one_month_data_2020.append(retrn_one_month)
    
    retrn_three_month=path_3_month_data_2020.loc[(path_3_month_data_2020['Ticker']==ticker) & (path_3_month_data_2020['Month']=='2020-03-01')]['Median'].iloc[0] -  path_3_month_data_2020.loc[(path_3_month_data_2020['Ticker']==ticker) & (path_3_month_data_2020['Month']=='2020-01-01')]['Median'].iloc[0]
    past_three_month_data_2020.append(retrn_three_month)
    
    retrn_one_year=path_1_year_data_2020.loc[(path_1_year_data_2020['Ticker']==ticker) & (path_1_year_data_2020['Month']=='2020-12-01')]['Median'].iloc[0] -  path_1_year_data_2020.loc[(path_1_year_data_2020['Ticker']==ticker) & (path_1_year_data_2020['Month']=='2020-01-01')]['Median'].iloc[0]
    past_one_year_data_2020.append(retrn_one_year)
    
    # label is the med_close at Jan-2021 (assume that I am now at Dec-2020 and wants to predict Jan-2021)
    retrn_jan_2021=past_january_2021.loc[(past_january_2021['Ticker']==ticker) & (past_january_2021['Month']=='2021-01-01')]['Median'].iloc[0]
    label.append(retrn_jan_2021)

testing_data=pd.DataFrame({
    'Ticker':Ticker,
    'past_one_month_data_2020':past_one_month_data_2020,   
    'past_three_month_data_2020':past_three_month_data_2020,
    'past_one_year_data_2020':past_one_year_data_2020,
    'label':label
    })
training_data
training_data.to_csv('training_data.csv')